See if we can extract a FITS image and apply some external code to it. 

In [ ]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry

config = '/repo/dc2/'
collection = '2.2i/runs/DP0.1'
butler = Butler(config, collections=collection)

registry = butler.registry

collections = registry.queryCollections("2.2i*DP0*")

for c in collections :
    print(c)


In [ ]:
datasetRefs = list(registry.queryDatasets(datasetType='calexp',collections=['2.2i/runs/DP0.1/calexp/v1']))
count = 0

for ref in datasetRefs:
    count = count + 1
    print(ref.dataId)
    if (count > 10): 
        break


Listing the images you want will have to be done with butler
Using one of the tutorial queries  -

In [ ]:
dataId = {'visit': 733724, 'detector': 175, 'band': 'i,r,g'}
calexp = butler.get('calexp', **dataId)
print(calexp)

That was a bit specific we can get a broader set ...

In [4]:
datasetType = 'calexp'
dataId = {'visit': 733724}
datasetRefs = set(registry.queryDatasets(datasetType, dataId=dataId))

print(f"Found {len(datasetRefs)} detectors")

Found 189 detectors


Now we have 1 or more images but apparently our code wants a FITS file not a Python object. 
We really should not be using old file based tools but if we must .. 
First lets look at this one calexp .. you do not need this following cell it is just in case you want to chekc the image.

In [ ]:
import lsst.afw.display as afwDisplay

display = afwDisplay.getDisplay(backend='firefly')

display.mtv(calexp)



In [ ]:


for count, exp in enumerate(datasetRefs):
    fn=f"Rubin-calexp-{exp.dataId['visit']}-{exp.dataId['detector']}.fits"
    exp.writeFits(fn)
    # just one for now
    #now do our thing ..
    break

print (f"{count} exposures ")



In [41]:
#  Astropy fits header look
from astropy.io import fits
fits_image_filename='../ext-soft-demo/Rubin-733724-175-calexp.fits'
hdul = fits.open(fits_image_filename)  


In [ ]:
for count,hdu in enumerate(hdul):
    if count <= 3:
        keys = hdu.header.keys()
        print (hdu.name)
        #  hdu.header['TFORM']="T_DOUBLE"
        for kcount,k in enumerate(keys):
            if k.startswith("TFOR"):
               print(f"{k}:{hdu.header[k]}")

        print(f" saw {kcount} keys")


In [44]:
hdul.writeto(fits_image_filename,overwrite=True)